In [1]:
import pandas as pd
import os

path=os.getcwd()

train_df = pd.read_csv(path + f'/데이터들/라벨 인코딩/train.csv')
test_df = pd.read_csv(path + f'/데이터들/라벨 인코딩/test.csv')

In [2]:
from flaml import AutoML
automl = AutoML()

# Specify your task and the metric to optimize for
automl_settings = {
    "time_budget": 300,  # time budget in seconds
    "metric": 'rmse',
    "task": 'regression',
    "log_file_name": "flaml_log.txt",
}

X_train=train_df.drop(['ECLO', 'Unnamed: 0', '사망자수', '중상자수', '경상자수', '부상자수', '요일', '기상상태', '노면상태', '사고유형', '시', '구', '동', '도로형태1', '도로형태2', 'holiday', '시간구분', '계절'], axis=1)

print(X_train.columns)

Index(['년도', '월', '시간', '기상상태_레이블', '노면상태_레이블', '사고유형_레이블', '시_레이블', '구_레이블',
       '동_레이블', '도로형태1_레이블', '도로형태2_레이블', 'holiday_레이블', '시간구분_레이블', '계절_레이블'],
      dtype='object')


In [3]:
# Train the model
automl.fit(X_train, y_train=train_df['ECLO'], **automl_settings)

# Predictions
flaml_predictions = automl.predict(test_df.drop(['Unnamed: 0'], axis=1))
print(flaml_predictions[:5])

[flaml.automl.logger: 02-27 17:22:02] {1679} INFO - task = regression
[flaml.automl.logger: 02-27 17:22:02] {1690} INFO - Evaluation method: holdout
[flaml.automl.logger: 02-27 17:22:02] {1788} INFO - Minimizing error metric: rmse
[flaml.automl.logger: 02-27 17:22:02] {1900} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl.logger: 02-27 17:22:02] {2218} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 02-27 17:22:03] {2344} INFO - Estimated sufficient time budget=23815s. Estimated necessary time budget=204s.
[flaml.automl.logger: 02-27 17:22:03] {2391} INFO -  at 1.0s,	estimator lgbm's best error=3.1479,	best estimator lgbm's best error=3.1479
[flaml.automl.logger: 02-27 17:22:03] {2218} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 02-27 17:22:03] {2391} INFO -  at 1.1s,	estimator lgbm's best error=3.1479,	best estimator lgbm's best error=3.1479
[flaml.automl.logger: 02-27 17:22

In [ ]:
submission_df = pd.DataFrame({
    'ID': test_df['ID'],  # Ensure this matches the ID column name in your test dataset
    'ECLO': flaml_predictions  # This is the array of predictions from FLAML
})

submission_file_path = path + f'/데이터들/전처리 전/submission_flaml.csv'

submission_df.to_csv(submission_file_path, index=False)